# Data integration (Ukrainian Vehicles Data Set)

### Loading the data

In [9]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import csv
from sklearn.metrics import mean_squared_error

%matplotlib inline

Let us load the data as a pandas dataframe.

In [10]:
data = pd.read_csv("cars_data.csv", sep = ";")
len(data)

1417655

### Exploring the data
Now take a look at the dataset and its features:

In [11]:
data.head(5)

,person,reg_addr_koatuu,oper_code,oper_name,d_reg,dep_code,dep,brand,model,make_year,color,kind,body,purpose,fuel,capacity,own_weight,total_weight,n_reg_new
0,P,2.310136e+09,530,530 - ЗНЯТТЯ З ОБЛIКУ ДЛЯ РЕАЛIЗАЦIЇ,2017-01-03,2341,Центр 2341,ВАЗ 210700-20,210700-20,2008,ЗЕЛЕНИЙ,ЛЕГКОВИЙ,СЕДАН-B,ЗАГАЛЬНИЙ,БЕНЗИН,1451.0,1060.0,1460.0,08ВА8008
1,P,1.210138e+09,99,99 - ПЕРВИННА РЕЄСТРАЦІЯ ТЗ ПРИДБАНОГО В ТОРГІ...,2017-01-03,1248,Центр 1248,ПА 004,004,2015,ЗЕЛЕНИЙ,ПРИЧІП,ПР-ЛЕГКОВИЙ-B,ЗАГАЛЬНИЙ,NaN,NaN,190.0,700.0,АЕ3999ХМ
2,P,7.101000e+08,315,315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГО...,2017-01-03,741,Центр 0741,SCHMITZ SPR 24L,SPR 24L,2003,ЧЕРВОНИЙ,НАПІВПРИЧІП,Н/ПР-БОРТОВИЙ-E,ЗАГАЛЬНИЙ,NaN,NaN,6694.0,35000.0,АС3826ХТ
3,P,7.101000e+08,315,315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГО...,2017-01-03,741,Центр 0741,RENAULT PREMIUM 420,PREMIUM 420,2004,БІЛИЙ,ВАНТАЖНИЙ,СІДЛОВИЙ ТЯГАЧ-E,СПЕЦІАЛІЗОВАНИЙ,ДИЗЕЛЬНЕ ПАЛИВО,11116.0,10880.0,24000.0,АС4142ВЕ
4,P,7.101000e+08,315,315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГО...,2017-01-03,741,Центр 0741,WIELTON PC16K,PC16K,2004,СІРИЙ,ПРИЧІП,ПР-БОРТОВИЙ-ТЕНТОВАНИЙ,ЗАГАЛЬНИЙ,NaN,NaN,5050.0,18000.0,АС0397ХТ


In [12]:
data.dtypes

person              object
reg_addr_koatuu    float64
oper_code            int64
oper_name           object
d_reg               object
dep_code             int64
dep                 object
brand               object
model               object
make_year            int64
color               object
kind                object
body                object
purpose             object
fuel                object
capacity           float64
own_weight         float64
total_weight       float64
n_reg_new           object
dtype: object

### Clean the data: 
- remove quotes
- trailing whitespaces
- tabs


In [13]:
import re

def trimAllColumns(df):
    """
    Trim whitespace, tabs and \n from ends of each value across all series in dataframe
    """
    trimStrings_1 = lambda x: re.sub('\s+', ' ', x) if type(x) is str else x
    trimStrings_2 = lambda x: x.strip() if type(x) is str else x
    df = df.applymap(trimStrings_1)
    return df.applymap(trimStrings_2)


# simple example of trimming whitespace from data elements
data = trimAllColumns(data)
data.head()

,person,reg_addr_koatuu,oper_code,oper_name,d_reg,dep_code,dep,brand,model,make_year,color,kind,body,purpose,fuel,capacity,own_weight,total_weight,n_reg_new
0,P,2.310136e+09,530,530 - ЗНЯТТЯ З ОБЛIКУ ДЛЯ РЕАЛIЗАЦIЇ,2017-01-03,2341,Центр 2341,ВАЗ 210700-20,210700-20,2008,ЗЕЛЕНИЙ,ЛЕГКОВИЙ,СЕДАН-B,ЗАГАЛЬНИЙ,БЕНЗИН,1451.0,1060.0,1460.0,08ВА8008
1,P,1.210138e+09,99,99 - ПЕРВИННА РЕЄСТРАЦІЯ ТЗ ПРИДБАНОГО В ТОРГІ...,2017-01-03,1248,Центр 1248,ПА 004,004,2015,ЗЕЛЕНИЙ,ПРИЧІП,ПР-ЛЕГКОВИЙ-B,ЗАГАЛЬНИЙ,NaN,NaN,190.0,700.0,АЕ3999ХМ
2,P,7.101000e+08,315,315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГО...,2017-01-03,741,Центр 0741,SCHMITZ SPR 24L,SPR 24L,2003,ЧЕРВОНИЙ,НАПІВПРИЧІП,Н/ПР-БОРТОВИЙ-E,ЗАГАЛЬНИЙ,NaN,NaN,6694.0,35000.0,АС3826ХТ
3,P,7.101000e+08,315,315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГО...,2017-01-03,741,Центр 0741,RENAULT PREMIUM 420,PREMIUM 420,2004,БІЛИЙ,ВАНТАЖНИЙ,СІДЛОВИЙ ТЯГАЧ-E,СПЕЦІАЛІЗОВАНИЙ,ДИЗЕЛЬНЕ ПАЛИВО,11116.0,10880.0,24000.0,АС4142ВЕ
4,P,7.101000e+08,315,315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГО...,2017-01-03,741,Центр 0741,WIELTON PC16K,PC16K,2004,СІРИЙ,ПРИЧІП,ПР-БОРТОВИЙ-ТЕНТОВАНИЙ,ЗАГАЛЬНИЙ,NaN,NaN,5050.0,18000.0,АС0397ХТ


### Find and remove duplicates (if any)

In [14]:
data.drop_duplicates(subset=['n_reg_new', 'oper_code'], keep=False)

,person,reg_addr_koatuu,oper_code,oper_name,d_reg,dep_code,dep,brand,model,make_year,color,kind,body,purpose,fuel,capacity,own_weight,total_weight,n_reg_new
0,P,2.310136e+09,530,530 - ЗНЯТТЯ З ОБЛIКУ ДЛЯ РЕАЛIЗАЦIЇ,2017-01-03,2341,Центр 2341,ВАЗ 210700-20,210700-20,2008,ЗЕЛЕНИЙ,ЛЕГКОВИЙ,СЕДАН-B,ЗАГАЛЬНИЙ,БЕНЗИН,1451.0,1060.0,1460.0,08ВА8008
1,P,1.210138e+09,99,99 - ПЕРВИННА РЕЄСТРАЦІЯ ТЗ ПРИДБАНОГО В ТОРГІ...,2017-01-03,1248,Центр 1248,ПА 004,004,2015,ЗЕЛЕНИЙ,ПРИЧІП,ПР-ЛЕГКОВИЙ-B,ЗАГАЛЬНИЙ,NaN,NaN,190.0,700.0,АЕ3999ХМ
2,P,7.101000e+08,315,315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГО...,2017-01-03,741,Центр 0741,SCHMITZ SPR 24L,SPR 24L,2003,ЧЕРВОНИЙ,НАПІВПРИЧІП,Н/ПР-БОРТОВИЙ-E,ЗАГАЛЬНИЙ,NaN,NaN,6694.0,35000.0,АС3826ХТ
3,P,7.101000e+08,315,315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГО...,2017-01-03,741,Центр 0741,RENAULT PREMIUM 420,PREMIUM 420,2004,БІЛИЙ,ВАНТАЖНИЙ,СІДЛОВИЙ ТЯГАЧ-E,СПЕЦІАЛІЗОВАНИЙ,ДИЗЕЛЬНЕ ПАЛИВО,11116.0,10880.0,24000.0,АС4142ВЕ
4,P,7.101000e+08,315,315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГО...,2017-01-03,741,Центр 0741,WIELTON PC16K,PC16K,2004,СІРИЙ,ПРИЧІП,ПР-БОРТОВИЙ-ТЕНТОВАНИЙ,ЗАГАЛЬНИЙ,NaN,NaN,5050.0,18000.0,АС0397ХТ
5,P,7.101000e+08,315,315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГО...,2017-01-03,741,Центр 0741,RENAULT PREMIUM 420,PREMIUM 420,2005,БІЛИЙ,ВАНТАЖНИЙ,СІДЛОВИЙ ТЯГАЧ-E,СПЕЦІАЛІЗОВАНИЙ,ДИЗЕЛЬНЕ ПАЛИВО,11116.0,6666.0,19000.0,АС4892ВН
6,J,5.124755e+09,319,319 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВОГО ВЛАСНИКА ЗА ...,2017-01-03,5145,Центр 5145,УАЗ 3962,3962,1991,ЗЕЛЕНИЙ,ЛЕГКОВИЙ,МЕДДОПОМОГА-B,СПЕЦІАЛІЗОВАНИЙ,БЕНЗИН,2445.0,1780.0,2740.0,ВН2433НН
7,J,5.124755e+09,319,319 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВОГО ВЛАСНИКА ЗА ...,2017-01-03,5145,Центр 5145,УАЗ 3152-01,3152-01,1990,ЖОВТИЙ,ЛЕГКОВИЙ,МЕДДОПОМОГА-B,СПЕЦІАЛІЗОВАНИЙ,БЕНЗИН,2445.0,1710.0,2540.0,ВН2429НН
8,J,5.124755e+09,319,319 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВОГО ВЛАСНИКА ЗА ...,2017-01-03,5145,Центр 5145,УАЗ 396294,396294,2007,БІЛИЙ,ЛЕГКОВИЙ,МЕДДОПОМОГА-B,СПЕЦІАЛІЗОВАНИЙ,БЕНЗИН,2890.0,1875.0,2720.0,ВН2905ВК
9,J,5.124755e+09,319,319 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВОГО ВЛАСНИКА ЗА ...,2017-01-03,5145,Центр 5145,УАЗ 3152-01,3152-01,1987,ЗЕЛЕНИЙ,ЛЕГКОВИЙ,МЕДДОПОМОГА-B,СПЕЦІАЛІЗОВАНИЙ,БЕНЗИН,2445.0,1710.0,2540.0,ВН2432НН


In [15]:
len(data)

1417655

#### length of data is the same as it was in cell 2, so there were no duplicates

### Save a dictionary of unique operation codes and its meanings in external file (op_codes.txt)

sorry I have troubles with encoding.
gonna skip this task.


In [16]:
import binascii
# oper_names = binascii.unhexlify(data['oper_name'].unique())

oper_names = data["oper_name"]

f = open("test.txt","w")

ops = {}

i = 0
for n in oper_names:
    ops[i] = n
    print(str(n))
    i += 1
    f.write(str(n))
    if i > 15: 
        break
f.close()

# oper_names = oper_names.unique()
# oper_names = oper_names.decode("utf-8")


# a = [binascii.unhexlify(oper_name)
#      for oper_name in oper_names]

# print(a)

# operations = {}
# for oper_name in oper_names:
#     curr_num = int(oper_name.split()[0])
#     if curr_num in operations: 
#         pass
#     else:
#         operations[curr_num] = binascii.unhexlify(binascii.hexlify(" ".join(oper_name.split()[2:])))
        
# print(str(operations))

530 - ЗНЯТТЯ З ОБЛIКУ ДЛЯ РЕАЛIЗАЦIЇ
99 - ПЕРВИННА РЕЄСТРАЦІЯ ТЗ ПРИДБАНОГО В ТОРГІВЕЛЬНІЙ ОРГАНІЗАЦІЇ, ЯКИЙ ВИГОТОВЛЕНО В УКРАЇНІ
315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ УКЛАДЕНОМУ В ТСЦ
315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ УКЛАДЕНОМУ В ТСЦ
315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ УКЛАДЕНОМУ В ТСЦ
315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ УКЛАДЕНОМУ В ТСЦ
319 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВОГО ВЛАСНИКА ЗА АКТОМ ПРИЙОМУ-ПЕРЕДАЧІ
319 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВОГО ВЛАСНИКА ЗА АКТОМ ПРИЙОМУ-ПЕРЕДАЧІ
319 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВОГО ВЛАСНИКА ЗА АКТОМ ПРИЙОМУ-ПЕРЕДАЧІ
319 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВОГО ВЛАСНИКА ЗА АКТОМ ПРИЙОМУ-ПЕРЕДАЧІ
319 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВОГО ВЛАСНИКА ЗА АКТОМ ПРИЙОМУ-ПЕРЕДАЧІ
319 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВОГО ВЛАСНИКА ЗА АКТОМ ПРИЙОМУ-ПЕРЕДАЧІ
540 - ЗНЯТТЯ З ОБЛIКУ У ЗВ`ЯЗКУ З ВИБРАКОВКОЮ
17 - ПЕРВИННА РЕЄСТРАЦІЯ ТЗ (МОПЕДІВ) ЗА ВИСНОВКАМИ
100 - ПЕРВИННА РЕЄСТРАЦIЯ ТЗ ПРИДБАНОГО В ТОРГІВЕЛЬНІЙ ОРГАНІЗАЦІЇ, ЯКИЙ ВВЕЗЕНО З-ЗА

In [17]:
with open('test.txt', 'r') as fi:
    lines = fi.readlines()[0]
    
print(lines)


530 - ЗНЯТТЯ З ОБЛIКУ ДЛЯ РЕАЛIЗАЦIЇ99 - ПЕРВИННА РЕЄСТРАЦІЯ ТЗ ПРИДБАНОГО В ТОРГІВЕЛЬНІЙ ОРГАНІЗАЦІЇ, ЯКИЙ ВИГОТОВЛЕНО В УКРАЇНІ315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ УКЛАДЕНОМУ В ТСЦ315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ УКЛАДЕНОМУ В ТСЦ315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ УКЛАДЕНОМУ В ТСЦ315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ УКЛАДЕНОМУ В ТСЦ319 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВОГО ВЛАСНИКА ЗА АКТОМ ПРИЙОМУ-ПЕРЕДАЧІ319 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВОГО ВЛАСНИКА ЗА АКТОМ ПРИЙОМУ-ПЕРЕДАЧІ319 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВОГО ВЛАСНИКА ЗА АКТОМ ПРИЙОМУ-ПЕРЕДАЧІ319 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВОГО ВЛАСНИКА ЗА АКТОМ ПРИЙОМУ-ПЕРЕДАЧІ319 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВОГО ВЛАСНИКА ЗА АКТОМ ПРИЙОМУ-ПЕРЕДАЧІ319 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВОГО ВЛАСНИКА ЗА АКТОМ ПРИЙОМУ-ПЕРЕДАЧІ540 - ЗНЯТТЯ З ОБЛIКУ У ЗВ`ЯЗКУ З ВИБРАКОВКОЮ17 - ПЕРВИННА РЕЄСТРАЦІЯ ТЗ (МОПЕДІВ) ЗА ВИСНОВКАМИ100 - ПЕРВИННА РЕЄСТРАЦIЯ ТЗ ПРИДБАНОГО В ТОРГІВЕЛЬНІЙ ОРГАНІЗАЦІЇ, ЯКИЙ ВВЕЗЕНО З-ЗА КОРДОНУ48 - В

In [18]:
data.head()

,person,reg_addr_koatuu,oper_code,oper_name,d_reg,dep_code,dep,brand,model,make_year,color,kind,body,purpose,fuel,capacity,own_weight,total_weight,n_reg_new
0,P,2.310136e+09,530,530 - ЗНЯТТЯ З ОБЛIКУ ДЛЯ РЕАЛIЗАЦIЇ,2017-01-03,2341,Центр 2341,ВАЗ 210700-20,210700-20,2008,ЗЕЛЕНИЙ,ЛЕГКОВИЙ,СЕДАН-B,ЗАГАЛЬНИЙ,БЕНЗИН,1451.0,1060.0,1460.0,08ВА8008
1,P,1.210138e+09,99,99 - ПЕРВИННА РЕЄСТРАЦІЯ ТЗ ПРИДБАНОГО В ТОРГІ...,2017-01-03,1248,Центр 1248,ПА 004,004,2015,ЗЕЛЕНИЙ,ПРИЧІП,ПР-ЛЕГКОВИЙ-B,ЗАГАЛЬНИЙ,NaN,NaN,190.0,700.0,АЕ3999ХМ
2,P,7.101000e+08,315,315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГО...,2017-01-03,741,Центр 0741,SCHMITZ SPR 24L,SPR 24L,2003,ЧЕРВОНИЙ,НАПІВПРИЧІП,Н/ПР-БОРТОВИЙ-E,ЗАГАЛЬНИЙ,NaN,NaN,6694.0,35000.0,АС3826ХТ
3,P,7.101000e+08,315,315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГО...,2017-01-03,741,Центр 0741,RENAULT PREMIUM 420,PREMIUM 420,2004,БІЛИЙ,ВАНТАЖНИЙ,СІДЛОВИЙ ТЯГАЧ-E,СПЕЦІАЛІЗОВАНИЙ,ДИЗЕЛЬНЕ ПАЛИВО,11116.0,10880.0,24000.0,АС4142ВЕ
4,P,7.101000e+08,315,315 - ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГО...,2017-01-03,741,Центр 0741,WIELTON PC16K,PC16K,2004,СІРИЙ,ПРИЧІП,ПР-БОРТОВИЙ-ТЕНТОВАНИЙ,ЗАГАЛЬНИЙ,NaN,NaN,5050.0,18000.0,АС0397ХТ


### Drop redundant columns.
If you find any, explain why you think of them being redundant?

- reg_addr_koatuu - redundant - wierd values, usless for analysis





In [19]:
data = data.drop(columns=["oper_name",
                        "reg_addr_koatuu"])

### Rename some values to English

In [20]:
data = data.replace({'БІЛИЙ': 'white',
 'БЕЖЕВИЙ': 'beige',
 'ЖОВТИЙ': 'yellow',
 'ЗЕЛЕНИЙ': 'green',
 'КОРИЧНЕВИЙ': 'brown',
 'НЕВИЗНАЧЕНИЙ': 'unknown',
 'ОРАНЖЕВИЙ': 'orange',
 'СІРИЙ': 'grey',
 'СИНІЙ': 'blue',
 'ФІОЛЕТОВИЙ': 'purple',
 'ЧЕРВОНИЙ': 'red',
 'ЧОРНИЙ': 'black'})

data = data.replace({'АВТОБУС': 'bus',
 'ВАНТАЖНИЙ': 'truck' ,
 'КВАДРОЦИКЛ': 'quadricycle' ,
 'ЛЕГКОВИЙ': 'car',
 'МОПЕД': 'motorbike',
 'МОТОЦИКЛ': 'motorcycle',
 'НАПІВПРИЧІП': 'semi-trailer',
 'ПРИЧІП': 'trailer',
 'СПЕЦІАЛІЗОВАНІ': 'special',
 'ТРИЦИКЛ': 'tricicle'})
data = data.replace({'ЗАГАЛЬНИЙ': 'general', 'СПЕЦІАЛІЗОВАНИЙ': 'specialized', 'СПЕЦІАЛЬНИЙ': 'speial'})

In [21]:
data.head()

,person,oper_code,d_reg,dep_code,dep,brand,model,make_year,color,kind,body,purpose,fuel,capacity,own_weight,total_weight,n_reg_new
0,P,530,2017-01-03,2341,Центр 2341,ВАЗ 210700-20,210700-20,2008,green,car,СЕДАН-B,general,БЕНЗИН,1451.0,1060.0,1460.0,08ВА8008
1,P,99,2017-01-03,1248,Центр 1248,ПА 004,004,2015,green,trailer,ПР-ЛЕГКОВИЙ-B,general,NaN,NaN,190.0,700.0,АЕ3999ХМ
2,P,315,2017-01-03,741,Центр 0741,SCHMITZ SPR 24L,SPR 24L,2003,red,semi-trailer,Н/ПР-БОРТОВИЙ-E,general,NaN,NaN,6694.0,35000.0,АС3826ХТ
3,P,315,2017-01-03,741,Центр 0741,RENAULT PREMIUM 420,PREMIUM 420,2004,white,truck,СІДЛОВИЙ ТЯГАЧ-E,specialized,ДИЗЕЛЬНЕ ПАЛИВО,11116.0,10880.0,24000.0,АС4142ВЕ
4,P,315,2017-01-03,741,Центр 0741,WIELTON PC16K,PC16K,2004,grey,trailer,ПР-БОРТОВИЙ-ТЕНТОВАНИЙ,general,NaN,NaN,5050.0,18000.0,АС0397ХТ


### Fix columns naming
For the simplicity let's fix the naming to a more logical one

In [22]:
data.rename(columns={'Unnamed': 'index',
                     'TI':'title',
                     'SO':'type',
                     'LeadAuthorFirst':'author_first_name',
                     'LeadAuthorLast':'author_last_name',
                     'EditorFirst':'editor_first_name',
                     'EditorLast':'editor_last_name',
                     'FirstInst2':'author_institution',
                     'Editor.Institution':'editor_institution',
                    }, 
                 inplace=True)

In [23]:
data.dtypes

person           object
oper_code         int64
d_reg            object
dep_code          int64
dep              object
brand            object
model            object
make_year         int64
color            object
kind             object
body             object
purpose          object
fuel             object
capacity        float64
own_weight      float64
total_weight    float64
n_reg_new        object
dtype: object

### What is the possible usage of this data? Give 2-3 examples?
1. Find the mean weight of the car in Ukraine to build better bridges, roads
2. Find the most popular car brand in Ukraine
3. Find the region in Ukraine with the biggest amount of cars, analysing lables from n_reg_new column (AA....)

1.8. Save the result of your work as a regular .csv file.

In [24]:
data.to_csv('result.csv', sep='\t', encoding='utf-8', index=True)

1.9. What is the most popular car and car color in Ukraine?

In [25]:
data['color'].value_counts().idxmax()

'grey'

# The end.